In [1]:
def Appointment(source,destination) :
    import pandas as pd
    raw_data=pd.read_csv(source)
    raw_data=raw_data[["CorporateName","PatientName","ApplicationId","ContactNo","PackageName","AppointmentDate","ApptTime","VisitType","ProviderName","ProviderState","ProviderCity","AppointmentStatus"]]
    import datetime as dt
    raw_data["ApptTime"]=pd.to_datetime(raw_data["ApptTime"],format="%H:%M")
    raw_data["ApptTime"]=raw_data["ApptTime"].dt.strftime("%H:%M")
    raw_data=raw_data.sort_values("ApptTime")
    raw_data.to_excel(destination,index=False)
    print("Enjoy Bro")

In [2]:
A="C:/Users/HP/Downloads/mis-data-1726490092674.csv"
B="C:/Users/HP/Downloads/dt.xlsx"

In [3]:
Appointment(A,B)

Enjoy Bro


In [7]:
def wip(source,destination):
    import pandas as pd
    import datetime as dt
    df_raw=pd.read_csv(source,usecols=["RequestDate", "PatientName", "ApplicationId", "AppointmentStatus", "LastCallStatus", "NumberofAttempts", "DND"])
    df=df_raw
    df["RequestDate"]=pd.to_datetime(df["RequestDate"],format="%d/%m/%Y")
    previous_financial_years_data_index=df[df["RequestDate"]<="31/03/2024"].index
    df=df.drop(previous_financial_years_data_index)
    completed_cases_index=df[df["AppointmentStatus"].isin(["QC Approved", "QC APPROVED", "Reports Uploaded", "Appointment Attended", "QC Rejected", "Sent For Interpretation"])].index
    df=df.drop(completed_cases_index)
    df.loc[df["NumberofAttempts"]>30,"Status"]="Max Attempts"
    dnd_cases_index=df[(df["Status"].isna()) & (df["DND"]=="Yes")].index
    df.loc[dnd_cases_index,"Status"]="DND"
    appointment_status_cases_index=df[(df["AppointmentStatus"].isin(["Cancelled", "Cancelled by insurer", "Appointment Confirmed", "Order sent to partner"])) & (df["Status"].isna())].index 
    df.loc[appointment_status_cases_index,"Status"] = df.loc[appointment_status_cases_index, "AppointmentStatus"]
    LastCallStatus_index=df[df["Status"].isna()].index 
    df.loc[LastCallStatus_index,"Status"]=df.loc[LastCallStatus_index,"LastCallStatus"]
    df["Status"]=df["Status"].fillna("Non Contactable")
    Working_On_It_index=df[df["Status"].isin(["Appointment Request Received", "Direct Medical", "Location Constraint", "Medical Done Report Awaited", "Reminder"])].index
    df.loc[Working_On_It_index,"Status"]="Working On It"
    workable_index=df[df["Status"].isin(["Appointment Confirmed", "Callback", "Non Contactable", "Order sent to partner", "Working On It"])].index
    df.loc[workable_index,"Type"]="Workable"
    df.loc[df[df["Type"].isna()].index,"Type"]="Non_Workable"
    df["RequestDate"]=df["RequestDate"].dt.strftime("%d/%m/%Y")
    df_status=df["Status"].value_counts()
    writer=pd.ExcelWriter(destination)
    df_status.to_excel(writer, sheet_name="Report")
    df_raw.to_excel(writer, sheet_name="Raw_Data",index=False)  
    df.to_excel(writer, sheet_name="Cleaned_Data",index=False)
    writer.close()
    print("Chalo aage ka apna dekh lo Bro")
    

In [8]:
C="C:/Users/HP/Downloads/mis-data-1725455828946.csv"
D="C:/Users/HP/Downloads/wip.xlsx"

In [9]:
wip(C,D)

Chalo aage ka apna dekh lo Bro


In [10]:
def report(source,destination):
    import pandas as pd
    import datetime as dt
    import numpy as np
    raw=pd.read_csv(source,usecols=["RequestDate", "PatientName", "ApplicationId","ReportUploaded", "AppointmentStatus", "LastCallStatus", "AppointmentDate", "QcApprovedDate", "NumberofAttempts", "DND"])
    group_case_index=raw[((raw["ApplicationId"].str.startswith("G"))&(raw["ApplicationId"].str.endswith("01")))|((raw["ApplicationId"].str.startswith(" G"))&(raw["ApplicationId"].str.endswith("01")))].index
    raw=raw.loc[group_case_index]
    raw["RequestDate"]=pd.to_datetime(raw["RequestDate"],format="%d/%m/%Y")
    raw["AppointmentDate"]=pd.to_datetime(raw["AppointmentDate"],format="%d/%m/%Y")
    raw["QcApprovedDate"]=pd.to_datetime(raw["QcApprovedDate"],format="%d/%m/%Y")
    previous_financial_years_data_index=raw[raw["RequestDate"]<="31/03/2024"].index
    raw=raw.drop(previous_financial_years_data_index)
    completed_cases_index=raw[raw["AppointmentStatus"].isin(["QC Approved", "QC APPROVED", "Reports Uploaded", "Appointment Attended", "QC Rejected", "Sent For Interpretation"])].index
    converted=raw.loc[completed_cases_index,["RequestDate", "PatientName", "ApplicationId","AppointmentDate","QcApprovedDate","ReportUploaded", "AppointmentStatus"]]
    converted["Appointment_TAT"]=(converted["AppointmentDate"]-converted["RequestDate"]).dt.days.astype("string")
    converted["QC_TAT"]=(converted["QcApprovedDate"]-converted["AppointmentDate"]).dt.days
    pd.to_numeric(converted["QC_TAT"],errors="coerce")
    for index in converted.index:
        if converted.loc[index,"Appointment_TAT"]=="0":
            converted.loc[index,"Appointment_TAT"]="T 0"
        elif converted.loc[index,"Appointment_TAT"]=="1":
            converted.loc[index,"Appointment_TAT"]="T + 1"
        elif converted.loc[index,"Appointment_TAT"]=="2":
            converted.loc[index,"Appointment_TAT"]="T + 2"
        elif converted.loc[index,"Appointment_TAT"]=="3":
            converted.loc[index,"Appointment_TAT"]="T + 3"
        elif converted.loc[index,"Appointment_TAT"]=="4":
            converted.loc[index,"Appointment_TAT"]="T + 4"
        else :
            converted.loc[index,"Appointment_TAT"]= "> T + 4"
    pending_cases_index=raw[~(raw["AppointmentStatus"].isin(["QC Approved", "QC APPROVED", "Reports Uploaded", "Appointment Attended", "QC Rejected", "Sent For Interpretation"]))].index
    pending=raw.loc[pending_cases_index,["RequestDate", "PatientName", "ApplicationId", "AppointmentStatus", "LastCallStatus", "NumberofAttempts", "DND"]]
    pending.loc[pending["NumberofAttempts"]>30,"Status"]="Max Attempts"
    dnd_cases_index=pending[(pending["Status"].isna()) & (pending["DND"]=="Yes")].index
    pending.loc[dnd_cases_index,"Status"]="DND"
    appointment_status_cases_index=pending[(pending["AppointmentStatus"].isin(["Cancelled", "Cancelled by insurer", "Appointment Confirmed", "Order sent to partner"])) & (pending["Status"].isna())].index
    pending.loc[appointment_status_cases_index,"Status"] = pending.loc[appointment_status_cases_index, "AppointmentStatus"]
    LastCallStatus_index=pending[pending["Status"].isna()].index
    pending.loc[LastCallStatus_index,"Status"]=pending.loc[LastCallStatus_index,"LastCallStatus"]
    pending["Status"]=pending["Status"].fillna("Non Contactable")
    Working_On_It_index=pending[pending["Status"].isin(["Appointment Request Received", "Direct Medical", "Location Constraint", "Medical Done Report Awaited", "Reminder"])].index
    pending.loc[Working_On_It_index,"Status"]="Working On It"
    workable_index=pending[pending["Status"].isin(["Appointment Confirmed", "Callback", "Non Contactable", "Order sent to partner", "Working On It"])].index
    pending.loc[workable_index,"Type"]="Workable"
    pending.loc[pending[pending["Type"].isna()].index,"Type"]="Non_Workable"
    Non_Workable_Data=pending.loc[pending["Type"]=="Non_Workable"]
    raw["AppointmentDate"]=raw["AppointmentDate"].dt.strftime("%d/%m/%Y")
    raw["QcApprovedDate"]=raw["QcApprovedDate"].dt.strftime("%d/%m/%Y")
    converted["AppointmentDate"]=converted["AppointmentDate"].dt.strftime("%d/%m/%Y")
    converted["QcApprovedDate"]=converted["QcApprovedDate"].dt.strftime("%d/%m/%Y")
    writer=pd.ExcelWriter(destination)
    raw.to_excel(writer, sheet_name="raw",index=False)
    converted.to_excel(writer, sheet_name="converted",index=False)
    pending.to_excel(writer, sheet_name="pending",index=False)
    Non_Workable_Data.to_excel(writer, sheet_name="Non_Workable_Data",index=False)
    writer.close()
    print("Bas yahi tak ab kuch to kar lo")

In [11]:
E="C:/Users/HP/Downloads/mis-data-1725032970062.csv"
F="C:/Users/HP/Downloads/report.xlsx"

In [12]:
report(E,F)

Bas yahi tak ab kuch to kar lo
